# Introduction to Partial Differential Equations
---

## Chapter 2: Elliptic PDEs, Poisson’s Equation, and a Two-Point Boundary Value Problem 
---

## Creative Commons License Information
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/80x15.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Introduction to Partial Differential Equations: Theory and Computations</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations" property="cc:attributionName" rel="cc:attributionURL">Troy Butler</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations" rel="dct:source">https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations</a>.

## Section 2.1:  Poisson’s Equation and a 2-point BVP
---

This section utilizes some basic calculus concepts. In particular, the [Fundamental Theorem of Calculus](https://en.wikipedia.org/wiki/Fundamental_theorem_of_calculus) and [integration by parts](https://en.wikipedia.org/wiki/Integration_by_parts) play crucial roles. You may want to review these prior to reading through this notebook.

**Key takeaways from this section:**

- Under mild assumptions, a unique solution exists to the 2-pt BVP.

- A Green's function is a useful tool that can be used to describe the solution, but it can be somewhat painful/annoying to derive in practice. We demonstrate how to implement an algorithmic approach to deriving Green's functions for two-point BVPs that utilizes a symbolic library in Python and some basic linear algebra.

- Properties of the source term $f$ impact the solution $u$. In particular, we see how nonnegativity and regularity of $f$ impact $u$. 

  A topic that is often studied in great detail in more advanced PDEs courses is the ***regularity of a solution.*** Regularity simply means smoothness. The main point we get at with regards to regularity is that a solution $u$ has at least two more derivatives than the source term $f$, so it is *smoother* than $f$. Of course, regularly can be impacted by complexities in geometry of the spatial domain as well as the regularity of any parameters that appear in the PDE as coefficients in the differential operator (but these are topics left for a more advanced PDEs course).

- The boundary conditions (BCs) can take on various forms (Dirichlet, Neumann, Robin, and mixed), which represent different types of physical conditions and can have significant impacts on the solution (e.g., whether a solution is unique or not).

---
### Section 2.1.1: Solutions and the Green's function for the two-point BVP</a>
---

Consider the two-point BVP 

$$
\large -u'' = f, \ x\in (a,b), \ u(a)=0=u(b).
$$

We seek a [Green's function $G=G(x,y)$](https://en.wikipedia.org/wiki/Green%27s_function) such that the solution to the BVP can be written compactly as

$$
\large u(x) = \int_a^b G(x,y)f(y)\, dy.
$$

---
#### So what is a Green's functions exactly?
---

The Wiki article https://en.wikipedia.org/wiki/Green's_function contains some useful information, but perhaps the most useful part of the article comes in the motivation where it states (edits and emphasis are my own):

> Thus, one may obtain the function $u(x)$ through knowledge of the Green's function ... and the [data]. This process relies upon the linearity of the [differential] operator ...
<br><br>
In other words, the solution ..., $u(x)$, can be determined by ... integration ... Although $f(x)$ is known, this integration cannot be performed unless $G$ is also known. The problem now lies in finding the Green's function $G$ ... For this reason, the Green's function is also sometimes called the ***fundamental solution*** ...
<br><br>
***Not every [differential] operator*** ... ***admits a Green's function***. A Green's function can also be thought of as a ***right inverse of [the differential operator]***. Aside from the difficulties of finding a Green's function for a particular operator, the integral [for computing $u(x)$] may be quite difficult to evaluate. However the method gives a theoretically exact result.

The integral used to compute $u(x)=\int_a^b G(x,y)f(y)\, dy$ in fact reveals something else about the Green's function $G(x,y)$. 
Specifically, the value of the solution $u$ at the point $x$ is determined by a type of ***weighted sum*** of the data $f$ over the interval $[a,b]$.
In a sense, *$G(x,y)$ is indicating how the value of the data $f$ at the point $y$ is influencing the solution $u$ at the point $x$.*

This can be described in more plain terms using a specific application for which this particular BVP above is a model.
Consider an elastic bar of length $b-a$ and uniform stiffness $1$ that is positioned horizontally with both ends clamped so that they cannot move to an external force applied to the bar.
Application of a force in the vertical direction causes a bending of the bar with vertical displacement modeled by the above BVP.
Suppose the force $f$ is given by

$$
\large d_{y,\Delta x}(x) = \begin{cases}
                                    \frac{x-y+\Delta x}{(\Delta x)^2}, & -\Delta x + y < x < y, \\
                                    -\frac{x-y-\Delta x}{(\Delta x)^2}, & y \leq x < y+\Delta x, \\
                                    0, & \text{else}.
                           \end{cases}
$$

- The $y$ in $d_{y, \Delta x}(x)$ denotes the location where the force is centered.

- The $\Delta x$ in $d_{y, \Delta x}(x)$ is used to denote the distance from $y$ for which the force is nonzero, i.e., the force is contained in $[y-\Delta x, y+\Delta x]$.

- Note that the peak value of the force is obtained when $x=y$ and is a value of $1/\Delta x$. 

- As $\Delta x\downarrow 0$, we see that the value of the force at $x=y$ increases to positive infinity. 

We show some plots of $d_{y,\Delta x}(x)$ below for $a=0$, $b=1$, $\Delta x = 0.1$ and $y=0.2, 0.5,$ and $0.7$. Students are encouraged to try different values here and also to create a function for the plotting and use an interactive widget to allow more seamless exploration of this $d_{y, \Delta x}(x)$ function.

In [ ]:
import numpy as np
%matplotlib widget  
import matplotlib.pyplot as plt

In [ ]:
def d_y(x, y, delta_x):
    if (-delta_x + y) < x < y:
        z = (x-y+delta_x)/(delta_x**2)
    elif (y <= x < y + delta_x):
        z = -(x-y-delta_x)/(delta_x**2)
    else:
        z = 0
    return z

In [ ]:
%matplotlib widget
plt.figure(0)

Delta_x = 0.1
ys = [0.2,0.5,0.7]
num_pts = 49  # number of interior points
x_mesh = np.linspace(0,1,num_pts+2)
f = np.zeros(num_pts+2)

for y in ys:
    for i in range(0,num_pts+2):
        f[i] = d_y(x_mesh[i], y, Delta_x)
    plt.plot(x_mesh, f, label=str(y))
plt.legend();

Using equations for the area of a triangle, we clearly see that 

$$
   \int_a^b d_{y,\Delta x}(x) \, dx = \int_{y-\Delta x}^{y+\Delta x} d_{y,\Delta x}\, dx = 1
$$ 

(assuming we did not choose $y$ too close to the boundary of $[a,b]$ which might truncate the support of this function).

Now, with such a forcing function (based on choosing an appropriate $y$ and $\Delta x$), we have that

$$
    \begin{eqnarray}
        u(x) &=& \int_a^b G(x,y) d_{y,\Delta x}(x)\, dy \\
             &=& \int_{y-\Delta x}^{y+\Delta x} G(x,y) d_{y,\Delta x}(x) \, dy.
    \end{eqnarray}
$$

Assuming that $G$ is also continuous, the integral mean value theorem implies there exists some $c\in(y-\Delta x, y+\Delta x)$ such that 

$$
    u(x) = G(x,c)d_{y,\Delta x}(c)(2\Delta x).
$$

Since $\int_a^b d_{y,\Delta x}(x)\, dx =  \int_{y-\Delta x}^{y+\Delta x} d_{y,\Delta x}\, dx = 1$, we expect that for sufficiently small $\Delta x$ that $d_{y,\Delta x}(c)(2\Delta x)\approx 1$ (again by the integral mean value theorem). 
Moreover, for small $\Delta x$, $c\approx y$.
Thus, for sufficiently small $\Delta x$, we expect that

$$
    u(x) \approx G(x,y).
$$

In other words, the Green's function describes how the bar bends at point $x$ due to a very localized unit force applied to the bar at point $y$. 

This suggests taking the limit as $\Delta x\downarrow 0$ of $d_{y,\Delta x}(x)$ and defining the limit function as $\delta_y(x)$, or simply as $\delta(x-y)$, which by pointwise limits appears to have the properties that

$$
    \delta(x-y)=0, \ \text{ if } x\neq y \Rightarrow \delta(x) = 0, \ \text{ if } x\neq 0,
$$

and

$$
    \delta(x-y) = \infty, \ \text{ if } x= y \Rightarrow \delta(x) = \infty, \ \text{ if } x\neq 0,
$$

and by the above integral approximations should also possess the properties that

$$
    \int_a^b \delta(x-y)\, dx = \left\{\begin{array}{rr}
                                        1, & \text{if } y\in[a,b],\\
                                        0, & \text{if } y\notin[a,b],
                                        \end{array}
                                 \right.
$$

and, for continuous functions $g$ on $[a,b]$, 

$$
    \int_a^b g(x)\delta(x-y)\, dx = \left\{\begin{array}{rr}
                                        g(y), & \text{if } y\in[a,b],\\
                                        0, & \text{if } y\notin[a,b].
                                        \end{array}
                                 \right.
$$

Of course, *no such function can have these properties.* 
Nonetheless, we refer to this $\delta$ as the Dirac delta function (see https://en.wikipedia.org/wiki/Dirac_delta_function), which is a type of *generalized* function where all the formal properties above are made quite rigorous using the theory of ***distributions*** (see https://en.wikipedia.org/wiki/Distribution_(mathematics)).
However, this topic is beyond the scope of this course (it is best studied in a more advanced PDEs or special topics course after taking functional analysis).
Therefore, we will simply use this Dirac delta function and all of its properties to derive formal solutions, Green's functions, etc. with the comforting knowledge that everything is somehow justified (similar to how we will use Fourier series later in the course).

---
## Section 2.1.2: An Algorithmic Approach to Determining the Green's Function
---

We use the [Theorem from the Wiki page](https://en.wikipedia.org/wiki/Green's_function#Theorem) to determine an algorithm for solving  

$$
\large -u'' = \delta(x-y), \ x\in(a,b), \ u(a)=0=u(b)
$$

where $y\in(a,b)$ and $\delta$ denotes the Dirac delta function. 
This solution determines the Green's function $G(x,y)$.

From conditions 1 and 2 in the Theorem, we get the following
> <span style='background:rgba(255,255,0, 0.5)'>Step 1: Determine continuous solutions to the homogeneous differential equation.</span>
<br><br>
The fundamental set of solutions is useful here, and the solution is broken up into two parts: where $x<y$ and where $y<x$.

When $x\neq y$, $\delta(x-y)=0$. The fundamental set of solutions to the homogeneous ODE $-u''=0$ is given by

$$
 \large   \{1, x\}. 
$$

Let $u^-(x)$ denote the part of the solution for $x<y$ and $u^+(x)$ denote the part of the solution for $x>y$. 

Then, we have that

$$
 \large u^-(x) = c_1 + c_2x,
$$

and

$$
\large u^+(x) = c_3 + c_4x,
$$

where $c_1,c_2,c_3, $ and $c_4$ are constants. 

From conditions 3 and 4 in the Theorem, we get the following
> <span style='background:rgba(255,255,0, 0.5)'>Step 2: Setup and solve a linear system to determine the constants.</span>
<br><br>
The boundary conditions and the fact that the derivative has to "jump" are the key things to use here to setup four conditions for the four unknown constants.

We use the following four conditions to setup a linear system of equations to determine the constants:

(1) $u^-(a) = 0$ *(This is the left BC, which applies to the $u^-$ function.)*

(2) $u^+(b) = 0$ *(This is the right BC, which applies to the $u^+$ function.)*

(3) $u^-(y) = u^+(y)$ *(This is the continuity criterion for the Green's function at the point $x=y$ in the interior of the domain.)*

(4) $(u^+)'(y) - (u^-)'(y) = -1$ *(This is the "jump" criterion for the derivative of the Green's function at the point $x=y$ in the interior of the domain.)*

It is conceptually convenient (but not necessary) to write these in the following order: (1), (3), (4), and (2). This represents a flow of information in the equations from left-to-right where the first equation represents the informatoin at $x=a$, the middle two equations represent the information at the interior point $x=y$, and the final equation represents the information at $x=b$. This gives the following system of equations.

$$
    \left\{
        \begin{array}{rrrrrrrrrr}
            c_1 &+& ac_2 & &  & &   &=& 0, \\
            -c_1 &-& yc_2 &+& c_3 &+& yc_4 &=& 0,\\
                & & -c_2  & &     &+& c_4  &=& -1, \\
                & &      & & c_3 &+& bc_4 &=& 0,
        \end{array}
    \right.
$$

which we solve below using the `sympy` library.

In [ ]:
import sympy as sp 

a, b, x, y = sp.symbols('a, b, x, y')

A = sp.Matrix(([1, a, 0, 0],
               [-1, -y, 1, y],
               [0, -1, 0, 1],
               [0, 0, 1, b]))

data = sp.Matrix([0,0,-1,0])

cs = sp.linsolve((A, data))  # sympy as a linsolve function to solve matrix-vector problems
print(cs)

In [ ]:
print(type(cs)) 

In [ ]:
cs  # Observe that this is a set, { }, containing a tuple, ( )

In [ ]:
cs.args[0]  # We access the components of the set using the args command to get the tuple

In [ ]:
# So maybe a better way to define the cs is to do the following
cs_pretty = sp.linsolve((A, data))
cs = cs_pretty.args[0]

In [ ]:
cs

In [ ]:
# The Green's function for x<y
sp.factor(sp.simplify(cs[0] +  cs[1]*x))

In [ ]:
# The Green's function for y\leq x
sp.factor(sp.simplify(cs[2] + cs[3]*x))

---
#### The Green's function for the 2-pt BVP
---

From the above computation, we see that

$$
    \large G(x,y) = \left\{ \begin{array}{rr}
                                \frac{1}{a-b}(a-x)(b-y), & x<y, \\
                                \frac{1}{a-b}(a-y)(b-x), & y\leq x.
                            \end{array}
                    \right.
$$

We often rewrite this as

$$
    \large G(x,y) = \left\{ \begin{array}{rr}
                                \frac{1}{b-a}(x-a)(b-y), & x<y, \\
                                \frac{1}{b-a}(y-a)(b-x), & y\leq x.
                            \end{array}
                    \right.
$$

We define this using the `Piecewise` function available in `sympy`. 

In [ ]:
# Defining the Green's function in the manner in which sympy defined it
G = sp.Piecewise(((a - x)*(b - y)/(a - b), x<y),
                 ((a - y)*(b - x)/(a - b), y<=x))

In [ ]:
print(G)

In [ ]:
G  # A visual check

We ``lambdify`` this function so that we can easily evaluate it and make plots to check that everything is working for a more familiar case when $a=0$ and $b=1$.

In [ ]:
from sympy.utilities.lambdify import lambdify

G_eval = lambdify((a,b,x,y), G)

In [ ]:
# Students are encouraged to try different values of a_example and b_example here
a_example = 0
b_example = 1

x_mesh = np.linspace(a_example, b_example, 201)

y_pt1 = 0.2*(b_example-a_example)+a_example  # y_pt1 is 20% of the way from a to b
y_pt2 = 0.5*(b_example-a_example)+a_example  # y_pt2 is 50% of the way from a to b
y_pt3 = 0.9*(b_example-a_example)+a_example  # y_pt3 is 90% of the way from a to b

%matplotlib widget
plt.figure(1)
plt.plot(x_mesh, G_eval(a_example,b_example,x_mesh,y_pt1), label='y_pt1')
plt.plot(x_mesh, G_eval(a_example,b_example,x_mesh,y_pt2), label='y_pt2')
plt.plot(x_mesh, G_eval(a_example,b_example,x_mesh,y_pt3), label='y_pt3')
plt.legend()

---
#### Student Activity
---

Use the Green's function derived above to determine solutions to the BVP when

(a) $f(x)=1$

(b) $f(x)=x$

(c) $f(x)=x^2$

Verify the functions $u(x)$ are indeed solutions to the BVP by direct substitution into the BVP. It may be simpler to first choose $a=0$ and $b=1$ to verify.

*I get you started by showing how to do this for $f(x)=1$ in Python and also "by hand" and leave the rest for students.*

In [ ]:
u1_indef = sp.integrate(G, y)
u1_indef

No surprise here in that the antiderivative is itself a piecewise function on $[a,b]$.

For $y\in [a, x]$, the antiderivative is given by the top function shown above and for $y\in[x,b]$ it is the bottom function shown above. 

Note that in integral form, solution $u(x)$ can be written as 

$$
    u(x) = \int_a^x G(x,y)f(y)\, dy + \int_x^b G(x,y)f(y)\, dy.
$$

The first integral involves $y\in [a,x]$ and the second integral involves $y\in[x,b]$. We use this knowledge below.

In [ ]:
u1_indef.args[0]  # The top part (funciton and condition as a tuple)

In [ ]:
u1_indef.args[0][0]  # The first condition function

In [ ]:
int_a_to_x = sp.simplify(u1_indef.args[0][0].subs(y,x) - u1_indef.args[0][0].subs(y,a))
int_a_to_x

In [ ]:
int_x_to_b = sp.simplify(u1_indef.args[1][0].subs(y,b) - u1_indef.args[1][0].subs(y,x))
int_x_to_b

In [ ]:
u1 = sp.simplify(int_a_to_x + int_x_to_b)
u1

In [ ]:
u1 = sp.factor(int_a_to_x + int_x_to_b)
print("Solution to part (a): u(x) = ")
u1

In [ ]:
# This is one way to substitute in values for a and b
u1_fcn = lambdify((a, b), u1) 
print("Check with a=0, b=1: u(x) = ")
u1_fcn(0,1)

*Students should be able to directly verify that $u(x)=\frac{1}{2}x(1-x)$ is indeed a solution to the BVP on $(0,1)$ and that the more general $u(x)$ given above is a solution to the BVP on $(a,b)$.*

<mark>Now the "by hand" solution to (a).</mark>

Before we do this, we notice that by definition of $G(x,y)$ we can expand and "simplify" (in a sense) the integral defining $u(x)$ as

$$
\begin{align}
    (b-a)u(x) &= (b-a)\int_a^b G(x,y)  f(y)\, dy \\ \\
         &= \underbrace{\int_a^x (y-a)(b-x)f(y)\, dy}_{\text{factor out $(b-x)$ since indep. of $y$.}} + \underbrace{\int_x^b (x-a)(b-y)f(y)\, dy}_{\text{factor out $(x-a)$ since indep. of $y$.}} \\ \\
         &= (b-x)\int_a^x (y-a)f(y)\, dy + (x-a)\int_x^b (b-y)f(y)\, dy \\ \\
\end{align}
$$

Now, with $f(x)=1$ (so $f(y)=1$), we substitute this into the above form and get

$$
\begin{align}
    (b-a)u(x) &= (b-x)\int_a^x (y-a)\, dy + (x-a)\int_x^b (b-y)\, dy \\ \\
              &= (b-x)\left[\frac{y^2}{2} - ay\right]_a^x + (x-a)\left[by - \frac{y^2}{2}\right]_x^b \\ \\
              &= (b-x)\left[\frac{x^2}{2} - ax - \frac{a^2}{2} + a^2\right] + (x-a)\left[b^2 - \frac{b^2}{2} - bx + \frac{x^2}{2}\right]
\end{align}.
$$

Ugh, now for algebraic mathemagic via `sympy` (don't forget to divide through by $(b-a)$ at the end).

In [ ]:
u_hand = ( (b-x) * (x**2/2-a*x-a**2/2+a**2) + (x-a) * (b**2-b**2/2-b*x+x**2/2) ) / (b-a)
u_hand

In [ ]:
sp.factor(sp.simplify(u_hand))

In [ ]:
u_hand.subs({a:0, b:1}).simplify()

In [ ]:
# Use this and new code cells to compute the solution to (b)

In [ ]:
# Use this and new code cells to compute the solution to (c)

---
### Section 2.1.3: Finding Green's Functions for Related 2-pt BVPs
---

What would the Green's function be if the PDE changed to $-u''+u = f(x)$?

For simplicity below, assume $a=0$ and $b=1$ and that homogeneous Dirichlet conditions are used. In that case, the fundamental set of solutions for $-u''+u=0$ is given by $\{e^{-x}, e^x\}$. Below, we present the code for obtaining the Green's function based on this fundamental set of solutions. 

In [ ]:
x, y = sp.symbols('x, y')

A = sp.Matrix(([1, 1, 0, 0],
               [-sp.exp(-y), -sp.exp(y), sp.exp(-y), sp.exp(y)],
               [sp.exp(-y), -sp.exp(y), -sp.exp(-y), sp.exp(y)],
               [0,0,sp.exp(-1),sp.exp(1)]))

data = sp.Matrix([0,0,-1,0])

cs_pretty = sp.linsolve((A, data))
cs_pretty

In [ ]:
cs = cs_pretty.args[0]
cs

In [ ]:
G = sp.Piecewise((cs[0]*sp.exp(-x)+cs[1]*sp.exp(x), x<y),
                 (cs[2]*sp.exp(-x)+cs[3]*sp.exp(x), y<=x))

In [ ]:
G.simplify()

In [ ]:
G_eval = lambdify((x,y), G)

In [ ]:
num_pts = 49
x_mesh = np.linspace(0, 1, num_pts + 2)

y_pt1 = 0.2
y_pt2 = 0.5
y_pt3 = 0.9

G_plot = 0*x_mesh

%matplotlib widget
plt.figure(2)

for i in range(num_pts+2):
    G_plot[i] = G_eval(x_mesh[i], y_pt1)   
plt.plot(x_mesh, G_plot, label='y_pt1')

for i in range(num_pts+2):
    G_plot[i] = G_eval(x_mesh[i], y_pt2)   
plt.plot(x_mesh, G_plot, label='y_pt2')

for i in range(num_pts+2):
    G_plot[i] = G_eval(x_mesh[i],y_pt3)   
plt.plot(x_mesh, G_plot, label='y_pt3')

plt.legend()

The plots above *look* similar to the previous plots associated with the Green's function for $-u''=f(x)$, but if you look closely enough, you will see there is a slight curvature to the "line" segments plotted above.

We explore this below in more detail.

In [ ]:
# The original Green's function for -u''=f
G1 = sp.Piecewise(((a - x)*(b - y)/(a - b), x<y),
                  ((a - y)*(b - x)/(a - b), y<=x))

In [ ]:
G_diff = sp.Piecewise((G.args[0][0] - G1.args[0][0].subs({a:0, b:1}), x<y),
                      (G.args[1][0] - G1.args[1][0].subs({a:0, b:1}), y<=x))
G_eval = lambdify((x,y), G_diff)

In [ ]:
num_pts = 49
x_mesh = np.linspace(0, 1, num_pts + 2)

y_pt1 = 0.2
y_pt2 = 0.5
y_pt3 = 0.9

G_plot = 0*x_mesh

%matplotlib widget
plt.figure(3)

for i in range(num_pts+2):
    G_plot[i] = G_eval(x_mesh[i], y_pt1)   
plt.plot(x_mesh, G_plot, label='y_pt1')

for i in range(num_pts+2):
    G_plot[i] = G_eval(x_mesh[i], y_pt2)   
plt.plot(x_mesh, G_plot, label='y_pt2')

for i in range(num_pts+2):
    G_plot[i] = G_eval(x_mesh[i],y_pt3)   
plt.plot(x_mesh, G_plot, label='y_pt3')

plt.legend()

What if the PDE was changed to $-u'' - u = f$?

In this case, the FSS is $\{\cos(x), \sin(x)\}$, so $u^-(x) = c_1\cos(x)+c_2\sin(x)$ and $u^+(x)=c_3\cos(x)+c_4\sin(x)$.

In [ ]:
x, y = sp.symbols('x, y')

A = sp.Matrix(([1, 0, 0, 0],
               [-sp.cos(y), -sp.sin(y), sp.cos(y), sp.sin(y)],
               [sp.sin(y), -sp.cos(y), -sp.sin(y), sp.cos(y)],
               [0, 0, sp.cos(1), sp.sin(1)]))

data = sp.Matrix([0,0,-1,0])

cs_pretty = sp.linsolve((A, data))
cs_pretty

In [ ]:
cs = cs_pretty.args[0]
cs

In [ ]:
G = sp.Piecewise((cs[0]*sp.cos(x)+cs[1]*sp.sin(x), x<y),
                 (cs[2]*sp.cos(x)+cs[3]*sp.sin(x), y<=x))

In [ ]:
G.simplify()

In [ ]:
G_eval = lambdify((x,y), G)

In [ ]:
num_pts = 49
x_mesh = np.linspace(0, 1, num_pts + 2)

y_pt1 = 0.2
y_pt2 = 0.5
y_pt3 = 0.9

G_plot = 0*x_mesh

%matplotlib widget
plt.figure(3)

for i in range(num_pts+2):
    G_plot[i] = G_eval(x_mesh[i], y_pt1)   
plt.plot(x_mesh, G_plot, label='y_pt1')

for i in range(num_pts+2):
    G_plot[i] = G_eval(x_mesh[i], y_pt2)   
plt.plot(x_mesh, G_plot, label='y_pt2')

for i in range(num_pts+2):
    G_plot[i] = G_eval(x_mesh[i],y_pt3)   
plt.plot(x_mesh, G_plot, label='y_pt3')

plt.legend()

---
### Section 2.1.4: Defining a `class` for Solving 2-pt BVPs
---

The process used above involved a lot of copy/paste and editing for the specific details that we kept changing between different PDEs. 

This process, quite frankly, is stupid. It is very likely that we are going to make errors by doing this. Instead, it is better to encode the process in a class. A user-defined function also does the trick, but considering that we are solving a *class of problems* with the Green's function, it makes sense to define a `class` of solvers.

If you need a quick tutorial (or a refresher) on the basics of Object Oriented Programming (OOP), then I suggest you check out this [notebook](https://github.com/CU-Denver-UQ/Math1376/blob/master/Lectures-and-Assignments/04-Computational-Applications/04-Computational-applications-lecture-prologue.ipynb).

In [ ]:
class solve_2pt_BVP_Dirichlet(object):
    def __init__(self, a, b, f1, f2):
        '''
        Initialize object for solving a 2pt BVP with homogeneous Dirichlet conditions.
        
        Parameters
        ----------
        a : float, int, or sympy.core.symbol.Symbol
            position of left-endpoint
        b : float, int, or sympy.core.symbol.Symbol
            position of right-endpoint
        f1 : A symbolic function of symbolic variable x
            first fundamental solution to homogeneous ODE
        f2 : A symbolic function of symbolic variable x
            second fundamental solution
        '''
        self.a = a
        self.b = b
        self.f1 = f1
        self.f2 = f2
        self.make_G()
        
    def make_G(self):
        '''
        Construct the Green's function.
        
        Note that we use the subs command to substitute in a, b, and y for x
        in the functions f1 and f2, as appropriate, for the various constraints 
        associated with constructing the Green's function.
        '''
        x, y = sp.symbols('x, y')

        A = sp.Matrix(([self.f1.subs({x:self.a}), self.f2.subs({x:self.a}), 0, 0],
                       [-self.f1.subs({x:y}), -self.f2.subs({x:y}), self.f1.subs({x:y}), self.f2.subs({x:y})],
                       [-sp.diff(self.f1).subs({x:y}), -sp.diff(self.f2).subs({x:y}), sp.diff(self.f1).subs({x:y}), sp.diff(self.f2).subs({x:y})],
                       [0, 0, self.f1.subs({x:self.b}), self.f2.subs({x:self.b})]))

        data = sp.Matrix([0,0,-1,0])

        cs = sp.linsolve((A, data)).args[0]
        
        self.G = sp.simplify( sp.Piecewise( (sp.factor(cs[0]*self.f1+cs[1]*self.f2), x<y),
                                            (sp.factor(cs[2]*self.f1+cs[3]*self.f2), y<=x) ) )
        
        self.G_eval = lambdify((x,y), self.G)
        
    def set_data_and_solve(self, f):
        '''
        Give a source term, f, and solve the problem. 
        Assume f is given as symbolic function of x. 
        Store this f as a new data attribute self.f. 
        
        We use the subs command to substitute y for x 
        to turn f into a symbolic function of y when
        integrating G(x,y)f(y) with respect to y.
        '''
        # Store f as a new data attribute
        self.f = f
        
        # Create symbolic variables x and y for the integration problems
        x, y = sp.symbols('x, y')
        
        # Now obtain the indefinite integral of G*f with respect to y
        # Note that the use of sp.factor can help break up the integrand 
        # into smaller, more manageable components, for the symbolic integrator
        u_form = sp.integrate(sp.factor(self.G*self.f), y)  
        
        # Now compute the definite integrals from a to x and from x to b.
        int_a_x = sp.simplify(u_form.args[0][0].subs(y, x) - u_form.args[0][0].subs(y, self.a))
        int_x_b = sp.simplify(u_form.args[1][0].subs(y, self.b) - u_form.args[1][0].subs(y, x))
        
        # Now add the definite integrals together to get u(x) and store as data attribute
        self.u = sp.factor(int_a_x+int_x_b)

----
#### Insantiate and use the class
---

Below, we check our class against results from the previous Student Activity.

In [ ]:
a, b, x, y = sp.symbols('a, b, x, y')  # Just in case these were overwritten above

In [ ]:
# One way to initialize solution object for the previos Student Activity
# which involves the 1st PDE, -u''=f, where we keep (a,b) arbitrary

solve_1st_pde = solve_2pt_BVP_Dirichlet(a, b, 1+0*x, x)  # Used 1+0*x to create symbolic constant 1 function 

In [ ]:
solve_1st_pde.G

In [ ]:
# One way to initialize solution object for the 2nd PDE, -u''+u=f with a=0 and b=1

solve_2nd_pde = solve_2pt_BVP_Dirichlet(0, 1, sp.exp(-x), sp.exp(x))

In [ ]:
solve_2nd_pde.G

In [ ]:
# One way to initialize solution object for the 3rd PDE, -u''-u=f with a=0 and b=1

solve_3rd_pde = solve_2pt_BVP_Dirichlet(0, 1, sp.cos(x), sp.sin(x))

In [ ]:
solve_3rd_pde.G

In [ ]:
solve_1st_pde.set_data_and_solve(1+0*y)  # The constant 1 function as a function of y

In [ ]:
solve_1st_pde.u

In [ ]:
sp.simplify(solve_1st_pde.u.subs({a: 0, b: 1}))  # Should be 0.5*x(1-x)

In [ ]:
# How to solve for some hard data
solve_1st_pde.set_data_and_solve(sp.exp(-y**2))

In [ ]:
sp.simplify(solve_1st_pde.u.subs({a: 0, b: 1}))  # Should be ??? Something nuts.

In [ ]:
solve_2nd_pde.set_data_and_solve(1+0*y)
sp.simplify(solve_2nd_pde.u)  # This is also going to be nuts!

In [ ]:
solve_3rd_pde.set_data_and_solve(1+0*y)
solve_3rd_pde.u  # Relatively speaking, this doesn't look too bad

In [ ]:
sp.simplify(solve_1st_pde.u.subs({a: 2, b: 3}))  # Can you imagine doing this by hand?

In [ ]:
# For a given a and b, this is how you turn u into an evaluatable function of x
u_eval = lambdify(x, solve_1st_pde.u.subs({a: 0, b: 10}))  

In [ ]:
u_eval(3)

---
#### Student Activity
---

If you really want to test your understanding of the theory and ability to code correctly, then here are two things to try. 

First, students are encouraged to generalize the class above to a `solve_2pt_BVP` superclass (also known as a base or parent class), and then sub-class a few solvers from this superclass based on different types of homogeneous boundary conditions (e.g., Robin, mixed-type, etc. although I would make sure that at least one of the boundary conditions was either Robin or Dirichlet to ensure the existence of a unique Green's function). 

Second, students are also encouraged to code a general `solve_2pt_BVP` that handles all types of boundary conditions based on the general algorthmic approach to constructing Green's functions as discussed at the top of this notebook when referencing the [Theorem from the Wiki page](https://en.wikipedia.org/wiki/Green's_function#Theorem). In other words, there are no sub-classes from this class. It is just a one-stop solver for all 2-point BVPs with homogeneous boundary conditions. *Hint: In this case, the `A` and `data` matrices constructed in the `make_G` method should probably involve some logic (i.e., conditionals) that handle how to construct these matrices based on the boundary types.*

Which approach to you prefer? Why?

*We will discuss the ideas of this activity in class, but the actual implementation is left as a non-required homework for the truly inspired student.*

---
### Section 2.1.5: Existence, Uniqueness, and Smoothness of Solutions</a>
---
<br>

We summarize the main result as Theorem 2.1.1 below. 

---
#### Theorem 2.1.1: Existence, Uniqueness and Smoothness (EUS). 

For every $f\in \mathcal{C}^k((a,b))$, there exists a unique solution $u\in\mathcal{C}^{k+2}((a,b))$ of the BVP 

$$
\begin{cases}
    -u'' &= f, \ x\in(a,b), \\
    u(a)=u(b) &= 0.
\end{cases}
$$

Furthermore, the solution $u$ admits the representation given by

$$
    u(x) = \int_a^b G(x,y)f(y)\, dy
$$

where $G(x,y)$ is given by

$$
    G(x,y) = \left\{ \begin{array}{rr}
                                \frac{1}{a-b}(a-x)(b-y), & x<y, \\
                                \frac{1}{a-b}(a-y)(b-x), & y\leq x.
                            \end{array}
                    \right.
$$

---

<br>

The proof is relatively straightforward. If we prove that $u(x)$ given in the form above is twice differentiable and satisfies the BVP, then we have that a solution exists in $\mathcal{C}^{k+2}((a,b))$. To prove uniqueness, we use the standard technique of assuming two solutions exist and showing that their difference must be zero.
<br><br>

---
#### Proof of Theorem 2.1.1:

Let 

$$
    u(x) = \int_a^b G(x,y)f(y)\, dy.
$$

We show that $u''$ exists and that this satisfies the BVP.

First, we differentiate $u$ with respect to $x$ and formally write $G_x$ which can be determined on the intervals $(a,x)$ and $(x,b)$ for which $y<x$ and $x<y$, respectively, to give

$$
\begin{align}
    u'(x) &= \int_a^b G_x(x,y)f(y)\, dy \\ \\
          &= -\frac{1}{a-b}\int_a^x  (a-y)f(y)\, dy - \frac{1}{a-b}\int_x^b (b-y)f(y)\, dy.
\end{align}
$$

Then, differentiating again with the Fundamental Theorem of Calculus gives

$$
\begin{align}
    u''(x) &= -\frac{1}{a-b}(a-x)f(x) + \frac{1}{a-b} (b-x)f(x) \\ \\
           &= \frac{f(x)}{a-b}\left[(b-x) - (a-x)\right] \\ \\ 
           &= -f(x).
\end{align}
$$

Thus, the given $u$ satisfies the PDE. Since $u''=-f(x)$ and $f\in\mathcal{C}^k((a,b))$, it follows that $u\in\mathcal{C}^{k+2}$, i.e., $u$ is twice as smooth as $f$.

To verify the BCs, we observe that if $x=0$, then $G(0,y) = 0$, so $u(0)=0$. Similarly, $G(1,y)=0$, so $u(1)=0$. Thus, $u$ satisfies both the PDE and the BCs so its a solution to the two-point BVP.

If $w$ is another solution to the two-point BVP and we let $v = u-w$, then it is straightforward to show that $v$ satisfies the two-point BVP

$$
    v''=0, \ x\in(a,b), \ v(a)=0=v(b).
$$

Solutions to $v''=0$ have the form $v(x)=c_1x+c_2$ for some constants $c_1$ and $c_2$. Using the BVs to determine these constants, we see that $c_1=c_2=0$, so $v(x)\equiv 0$, which implies that $u=w$. In other words, $u$ is the unique solution to the two-point BVP. 
$\Box$
---

---
### Section 2.1.6: Important Properties of Solutions $u$
---

The solution to 

$$
\begin{cases}
    -u'' &= f, \ x\in(a,b), \\
    u(a)=u(b) &= 0.
\end{cases}
$$

has two more important properties we discuss here.

<br>

---
#### Theorem 2.1.2: Monotonicity

Assume that $f\in\mathcal{C}((a,b))$ is a nonnegative function, then the solution $u\in\mathcal{C}^2((a,b))$ is also nonnegative.

---

<br>

---
#### Proof of Theorem 2.1.2:

Observe that $G(x,y)\geq 0$ for all $x,y\in[a,b]$, so this follows from the form of the solution $u$ given in Theorem 2.1.1 since the integral from $a$ to $b$ with $a<b$ of a nonnegative function is nonnegative. 
$\Box$
---

<br>

---
#### Theorem 2.1.3: Maximum principle

Assume that $f\in\mathcal{C}((a,b))$ and let $u$ be the unique solution of the BVP given in Theorem 2.1.1, then

$$
    \| u \|_\infty \leq \frac{(b-a)^2}{8}\|f\|_\infty.
$$

---

<br>

---
#### Proof of Theorem 2.1.3:

Let $x\in[a,b]$. Since $G(x,y)$ is nonnegative,

$$
    |u(x)| \leq \|f\|_\infty \int_a^b G(x,y)\, dy.
$$

Since

$$
    \int_a^b G(x,y)\, dy
$$

defines a parabola that opens downward for $x\in[a,b]$ that is equal to zero at $x=a$ and $x=b$, it has its maximum at $x=(b+a)/2$. Calculating this parabola at $x=(b+a)/2$ reveals that

$$
    \left\| \int_a^b G(x,y)\, dy \right\|_\infty = \frac{(b-a)^2}{8}.
$$

The conclusion follows. $\Box$

---

In the proof above, I used the Python code below to figure out the maximum value rather than do it by hand.

In [ ]:
solve_1st_pde.set_data_and_solve(1)

In [ ]:
sp.simplify(solve_1st_pde.u.subs({x: (b+a)/2}))

---
### Section 2.1.7: Pure Neumann BVP
---

Before we finish this notebook, we consider an alternative BVP with pure Neumann BCs given by

$$
    -u''(x) = f(x), \ x\in (a,b), \ u'(a)=u'(b)=0.
$$

---
#### Solvability
---

Suppose $u$ is a solution to this Neumann BVP, then $u''=-f$. The fundamental theorem of calculus implies that

$$
    u'(b)-u'(a) = \int_a^b -f(x)\, dx \Rightarrow \int_a^b f(x)\, dx = 0 \Rightarrow \frac{1}{b-a}\int_a^b f(x)\, dx.
$$

- In other words, if $u$ is a solution to this Neumann BVP, then necessarily the *average value of $f(x)$ is zero on $(a,b)$*. This is not saying that a solution exists if this condition holds, but rather it is saying that if the average value of $f(x)$ is *not* zero on $(a,b)$, then there is *no* solution to this problem.

- Consider the linear algebra analogy of $Ax=b$ being solvable if and only if $b$ is in the column space of $A$. Remember, we do not always expect solutions to problems to exist. There are usually certain solvability criteria that the data must satisfy. These are often referred to as "compatability criteria" to indicate that some data are simply incompatible with certain problems.

In what follows, we assume for the pure Neumann BVP that the average value of $f(x)$ is zero and determine if there is indeed a solution to this problem.

Before we do this, we address the uniqueness of solutions if they were to exist.

Observe that if $u$ is a solution, then so is $v(x)=u(x)+c$ for any $c\in\mathbb{R}$ (this can be verified by direct substitution). This implies that if a solution exists, then it is not unique.

- Consider the linear algebra analogy of $Ax=b$ for a square system where the matrix $A\in\mathbb{R}^{n\times n}$ is singular, i.e., the rank of $A$ is less than $n$ meaning it has a non-trivial nullspace. If $b\in\mathbb{R}^n$ satisfies the property that it is in the column space of $A$, solutions exist but they are non-unique because adding any vector in the nullspace of $A$ to the solution still produces a vector that $A$ maps to $b$. 

- Neumann BCs are used to model perfect insulation of the boundary in the heat equation, so this type of 2-pt BVP arises in modeling the steady-state solution to a heat equation where perfect insulation of the boundary occurs. Of course, we do not have the IC to serve as a Dirichlet BC in the space-time domain, and this loss of information makes the steady-state problem much like the $Ax=b$ problem discussed above where $A$ has a non-trivial nullspace. 

Returning to the idea that we can add a nonzero constant to a solution and still have a solution, note that the addition of a nonzero constant to any function fundamentally changes its average value on the interval $(a,b)$. This means that if we seek, for instance, a solution $u$ that has a specified average value on $(a,b)$, then the inclusion of this condition on the statement of the problem will lead to any solution being unique.

- For underdetermined problems $Ax=b$, we may seek a solution $x$ that, for instance, has the minimum norm. This is a way of "regularizing" the problem so that the solutions become unique under this additional criterion.

Let us suppose that we seek a solution $u$ such that $\int_a^b u(x)\, dx = 0$. Does such a solution exist?

We try to find a Green's function.

---
#### Attempt at finding a Green's function
---

The difference in constructing the Green's function for this pure Neumann problem as compared to the pure Dirichlet problem is initially only evident in the first two conditions below

(1) $(u^-)'(a) = 0$

(2) $(u^+)'(b) = 0$

(3) $u^-(y) = u^+(y)$

(4) $(u^+)'(y) - (u^-)'(y) = -1$

As before, we write these in the following order: (1), (3), (4), and (2) to represent a flow of information in the equations from left-to-right where the middle two equations represent the information at the interior point $y$. This gives the following system of equations.


$$
    \left\{
        \begin{array}{rrrrrrrrrr}
            0 &+& c_2 & &  & &   &=& 0, \\
            -c_1 &-& yc_2 &+& c_3 &+& yc_4 &=& 0,\\
                & & -c_2  & &     &+& c_4  &=& -1, \\
                & &      & & 0 &+& c_4 &=& 0,
        \end{array}
    \right.
$$

Looking at the first and fourth equations, we see $c_2=c_4=0$, yet $-c_2+c_4=-1$ according to the third equation. A contradiction! Well, I guess we are not going to be looking for a Green's function! Considering the extra conditions we needed on $f(x)$ for there to even exist the possibility of a solution, it is perhaps not surprising that a Green's function was not likely to exist.

---
#### Not giving up yet
---

Is there anything else we can try to solve this problem?

Well, suppose $u\in\mathcal{C}^2((a,b))$ solves the pure Neumann problem. Let's try to integrate the differential equation directly (two times) to get a function of $x$. We first integrate from $a$ to $y$ and then integrate from $a$ to $x$ to end up with a function of $x$. 

The first integration gives

$$
    u'(y)-\underbrace{u'(a)}_{=0} = \int_a^y -f(z)\, dz \ \Longrightarrow \ u'(y) = \int_a^y -f(z)\, dz = F(y).
$$

Here, we have rewritten $\int_a^y -f(z)\, dz = F(y)$ to emphasize that this definite integral defines a function of $y$. Integrating again with respect to $y$ from $a$ to $x$ gives

$$
    u(x)-u(a) = \int_a^x F(y)\, dy.
$$

How do we make sense of this integral on the right hand side? Let us try to apply integration by parts. This gives

$$
    \int_a^x F(y) \, dy = \left[yF(y)\right]_a^x - \int_a^x yF'(y)\, dy
$$

Substituting $F'(y) = -f(y)$ yields

$$
\begin{align}
    \int_0^x F(y) \, dy &= \left[-xF(x) + a(\underbrace{F(a)}_{=0})\right] + \int_a^x yf(y)\, dy \\
                        \\
                        &= -xF(x) + \int_a^x yf(y)\, dy \\ 
                        \\
                        &= -x\underbrace{\int_a^x f(z)\, dz}_{\text{Now change $z$ to $y$}} + \int_a^x yf(y)\, dy \\
                        \\
                        &= \int_a^x -xf(y)\, dy + \int_a^x yf(y)\, dy \\
                        \\
                        &= \int_a^x (y-x)f(y)\, dy.
\end{align}
$$

Therefore,

$$
    u(x) = u(a) + \int_a^x (y-x)f(y)\, dy.
$$

*Students should be able to verify that this form satisfies the pure Neumann BVP.*

Observe that $u(a)$ is just a constant, but it is involving $u(x)$ evaluated at $x=a$, which is quite unsatisfying. What should this constant be?

If we want $\int_a^b u(x)\, dx = 0$, then

$$
    0 = u(a)(b-a) + \int_a^b \int_a^x (y-x)f(y)\, dy\, dx \ \Longrightarrow \ u(a) = - \frac{1}{b-a} \int_a^b \int_a^x (y-x)f(y)\, dy\, dx,
$$

which means that this constant is uniquely determined by the average value of an integral involving the integral of the source term $f(x)$.

---
#### Student Activity
---

Use `sympy` to determine the solutions with zero average value to the 2-pt BVP $-u''=f$ on $(a, b)$ with pure Neumann BCs where 

(a) $f(x)=1$

(b) $f(x)=x$

(c) $f(x)=x^2$

---
#### Student Activity
---

Students are encouraged to reconsider the entire analysis of Section 2.1.7 onwards if $-u''=f$ is changed to $-u''-u = f$ or $-u''+u=f$. What steps change? What conclusions change? What analogies to linear systems of the form $Ax=b$ can we make? 

---
## Navigation:

- [Previous](Chp2Overview.ipynb)

- [Next](Chp2Sec2.ipynb)
---